In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-pbp/all_games.csv


In [2]:
import pandas as pd 

data = pd.read_csv('/kaggle/input/nba-pbp/all_games.csv')
data 

/tmp/ipykernel_24/4075445781.py:3: DtypeWarning: Columns (4,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/kaggle/input/nba-pbp/all_games.csv')


In [58]:
features = ['Period','Time', 'AwayName',
       'AwayScore', 'HomeName', 'HomeScore']

data = data[features]

In [85]:
data.tail(5)

,Period,Time,AwayName,AwayScore,HomeName,HomeScore
13151820,4,0:08.7,POR,91,NJN,87
13151821,4,0:08.7,POR,91,NJN,87
13151822,4,0:04.9,POR,91,NJN,87
13151823,4,0:03.7,POR,91,NJN,87
13151824,4,0:00.0,POR,91,NJN,87


In [87]:
# Assuming 'data' contains your dataset
df = pd.DataFrame(data)

# Create the 'HomeWin' and 'AwayWin' columns based on scores
df['HomeWin'] = (df['HomeScore'] > df['AwayScore']).astype(int)  # Home win (1) if HomeScore > AwayScore
df['AwayWin'] = (df['AwayScore'] > df['HomeScore']).astype(int)  # Away win (1) if AwayScore > HomeScore

# In case of a draw (both scores are equal), set both to 0
df.loc[df['HomeScore'] == df['AwayScore'], ['HomeWin', 'AwayWin']] = 0

# Display the DataFrame with the 'HomeWin' and 'AwayWin' columns
print(df)

          Period     Time AwayName AwayScore HomeName HomeScore  HomeWin  \
0              1  12:00.0      SAS         0      MIN         0        0   
1              1  12:00.0      SAS         0      MIN         0        0   
2              1  11:44.0      SAS         0      MIN         2        1   
3              1  11:30.0      SAS         2      MIN         2        0   
4              1  11:11.0      SAS         2      MIN         2        0   
...          ...      ...      ...       ...      ...       ...      ...   
13151820       4   0:08.7      POR        91      NJN        87        0   
13151821       4   0:08.7      POR        91      NJN        87        0   
13151822       4   0:04.9      POR        91      NJN        87        0   
13151823       4   0:03.7      POR        91      NJN        87        0   
13151824       4   0:00.0      POR        91      NJN        87        0   

          AwayWin  
0               0  
1               0  
2               0  
3      

In [88]:
# List of numeric team values that you want to remove
non_team_values = ['0', '2', '4', '6', '8', '10', '12', '14', '16', '18', '20', '21', '22', '24', 
                   '27', '29', '31', '33', '34', '37', '39', '41', '42', '44', '46', '48', '50', '53', 
                   '55', '57', '59', '61', '63', '65', '66', '67', '69', '72', '73', '74', '77', '78', 
                   '3', '5', '7', '9', '11', '13', '19', '23', '25', '30', '32', '35', '36', '43', '45', 
                   '47', '49', '51', '62', '64', '68', '71', '75', '76', '79', '81', '83', '84', '86', 
                   '87', '89', '90', '91', '92', '94', '95', '97', '98', '99', '100', '15', '17', '28', 
                   '38', '40', '52', '54', '56', '58', '60', '80', '82', '85', '88', '93', '101', '103', 
                   '105', '106', '107', '108', '109', '110', '111', '120', '121', '122', '123', '125', 
                   '127', '128', '130', '131', '126', '129', '124', '133', '135', '136']

# Remove rows where either AwayName or HomeName is in the non-team values list
df_cleaned = df[~df['AwayName'].isin(non_team_values) & ~df['HomeName'].isin(non_team_values)]

# Check how many rows were removed
removed_rows_count = len(df) - len(df_cleaned)

print(f"Removed {removed_rows_count} rows with non-team values.")


Removed 688339 rows with non-team values.


In [89]:
df.head()

,Period,Time,AwayName,AwayScore,HomeName,HomeScore,HomeWin,AwayWin
0,1,12:00.0,SAS,0,MIN,0,0,0
1,1,12:00.0,SAS,0,MIN,0,0,0
2,1,11:44.0,SAS,0,MIN,2,1,0
3,1,11:30.0,SAS,2,MIN,2,0,0
4,1,11:11.0,SAS,2,MIN,2,0,0


In [90]:
# Convert the 'Time' column to seconds
def convert_time_to_seconds(time_str):
    try:
        # The time format is like '12:00.0' (minute:second.millisecond)
        minutes, seconds = time_str.split(':')  # Split into minutes and seconds
        seconds, _ = seconds.split('.')  # Remove milliseconds (if any)
        return int(minutes) * 60 + int(seconds)
    except ValueError:
        # If there's an error in conversion, return NaN or handle it as needed
        return None

# Apply the function to the 'Time' column and create a new column 'TimeInSeconds'
df['TimeInSeconds'] = df['Time'].apply(convert_time_to_seconds)

# Drop rows where 'TimeInSeconds' is NaN (optional, based on your needs)
df = df.dropna(subset=['TimeInSeconds'])

# Calculate the 'TimeRemaining' using the formula: (4 - Period) * 12 * 60 + Time in seconds
df['TimeRemaining'] = (4 - df['Period']) * 12 * 60 + df['TimeInSeconds']

# Display the first few rows to check the result
print(df[['Period', 'Time', 'TimeRemaining']].head())


   Period     Time  TimeRemaining
0       1  12:00.0         2880.0
1       1  12:00.0         2880.0
2       1  11:44.0         2864.0
3       1  11:30.0         2850.0
4       1  11:11.0         2831.0


In [91]:
df = df[:400000]

In [92]:
df.columns

Index(['Period', 'Time', 'AwayName', 'AwayScore', 'HomeName', 'HomeScore',
       'HomeWin', 'AwayWin', 'TimeInSeconds', 'TimeRemaining'],
      dtype='object')

In [99]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assuming 'df' contains your dataset
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Encoding AwayName and HomeName columns
df['AwayName_encoded'] = label_encoder.fit_transform(df['AwayName'])
df['HomeName_encoded'] = label_encoder.fit_transform(df['HomeName'])

# Drop the original categorical columns
df_encoded = df.drop(['AwayName', 'HomeName', 'Time', 'Period'], axis=1)

# Prepare features (X) and target (y)
X = df_encoded.drop(['HomeWin', 'AwayWin'], axis=1)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, df_encoded[['HomeWin', 'AwayWin']], test_size=0.2, random_state=42)

# Initialize Logistic Regression models for HomeWin and AwayWin
model_home = LogisticRegression(random_state=42, max_iter=1000)
model_away = LogisticRegression(random_state=42, max_iter=1000)

# Train the Logistic Regression models for both HomeWin and AwayWin
model_home.fit(X_train, y_train['HomeWin'])
model_away.fit(X_train, y_train['AwayWin'])

# Predict on the test set
y_pred_home = model_home.predict(X_test)
y_pred_away = model_away.predict(X_test)

# Evaluate the models' accuracy
accuracy_home = accuracy_score(y_test['HomeWin'], y_pred_home)
accuracy_away = accuracy_score(y_test['AwayWin'], y_pred_away)
print(f"HomeWin Accuracy: {accuracy_home}")
print(f"AwayWin Accuracy: {accuracy_away}")

# Get the predicted probabilities for each class (HomeWin and AwayWin)
y_prob_home = model_home.predict_proba(X_test)
y_prob_away = model_away.predict_proba(X_test)

# The `predict_proba` method returns probabilities for both classes (0 and 1)
# y_prob_home[:, 1] will give you the probability for the 'HomeWin' class (1)
# y_prob_away[:, 1] will give you the probability for the 'AwayWin' class (1)

df_test_with_prob = X_test.copy()
df_test_with_prob['predicted_home_win_probability'] = y_prob_home[:, 1]  # Home win probability
df_test_with_prob['predicted_away_win_probability'] = y_prob_away[:, 1]  # Away win probability

# Map the encoded columns back to their original names (for both Away and Home teams)
df_test_with_prob['AwayName'] = label_encoder.inverse_transform(df_test_with_prob['AwayName_encoded'])
df_test_with_prob['HomeName'] = label_encoder.inverse_transform(df_test_with_prob['HomeName_encoded'])

# Format the output in the desired format
team_win_probabilities = df_test_with_prob[['AwayName', 'HomeName', 'predicted_home_win_probability', 'predicted_away_win_probability']]

# Display the table
team_win_probabilities


HomeWin Accuracy: 0.892425
AwayWin Accuracy: 0.8685625


,AwayName,HomeName,predicted_home_win_probability,predicted_away_win_probability
23624,IND,HOU,0.308255,0.634522
21137,CHA,IND,0.880986,0.092777
40449,CLE,TOR,0.936920,0.044008
153541,BRK,MIA,0.150659,0.756021
331152,POR,LAC,0.374448,0.589823
...,...,...,...,...
56267,OKC,TOR,0.846024,0.111360
103,SAS,MIN,0.890982,0.086287
358192,WAS,LAC,0.406485,0.506292
332614,NJN,MIL,0.048337,0.925637


In [101]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier

# Assuming 'df' contains your dataset
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Encoding AwayName and HomeName columns
df['AwayName_encoded'] = label_encoder.fit_transform(df['AwayName'])
df['HomeName_encoded'] = label_encoder.fit_transform(df['HomeName'])

# Drop the original categorical columns
df_encoded = df.drop(['AwayName', 'HomeName', 'Time', 'Period'], axis=1)

# Prepare features (X) and target (y)
X = df_encoded.drop(['HomeWin', 'AwayWin'], axis=1)
y = df_encoded[['HomeWin', 'AwayWin']]  # Target now includes both HomeWin and AwayWin

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize LogisticRegression model
log_reg_model = LogisticRegression(random_state=42, max_iter=1000)  # Increase max_iter if convergence warnings occur

# Wrap Logistic Regression in a MultiOutputClassifier
model = MultiOutputClassifier(log_reg_model)

# Train the Logistic Regression model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model's accuracy for both HomeWin and AwayWin
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Get the predicted probabilities for each class (HomeWin and AwayWin)
y_prob = model.predict_proba(X_test)

# The `predict_proba` method returns probabilities for both classes (0 and 1)
# y_prob[:, 1] will give you the probability for the 'win' class (1)
df_test_with_prob = X_test.copy()

# For each target column (HomeWin and AwayWin), you need to extract the probabilities for each target separately.
df_test_with_prob['predicted_home_win_probability'] = y_prob[0][:, 1]  # Home win probability
df_test_with_prob['predicted_away_win_probability'] = y_prob[1][:, 1]  # Away win probability

# Map the encoded columns back to their original names (for both Away and Home teams)
df_test_with_prob['AwayName'] = label_encoder.inverse_transform(df_test_with_prob['AwayName_encoded'])
df_test_with_prob['HomeName'] = label_encoder.inverse_transform(df_test_with_prob['HomeName_encoded'])

# Format the output in the desired format
team_win_probabilities = df_test_with_prob[['AwayName', 'HomeName', 'predicted_home_win_probability', 'predicted_away_win_probability']]

# Display the table
team_win_probabilities


Accuracy: 0.8479625


,AwayName,HomeName,predicted_home_win_probability,predicted_away_win_probability
23624,IND,HOU,0.308255,0.634522
21137,CHA,IND,0.880986,0.092777
40449,CLE,TOR,0.936920,0.044008
153541,BRK,MIA,0.150659,0.756021
331152,POR,LAC,0.374448,0.589823
...,...,...,...,...
56267,OKC,TOR,0.846024,0.111360
103,SAS,MIN,0.890982,0.086287
358192,WAS,LAC,0.406485,0.506292
332614,NJN,MIL,0.048337,0.925637


In [112]:
df.head()

,Period,Time,AwayName,AwayScore,HomeName,HomeScore,HomeWin,AwayWin,TimeInSeconds,TimeRemaining,AwayName_encoded,HomeName_encoded
0,1,12:00.0,SAS,0,MIN,0,0,0,720.0,2880.0,31,19
1,1,12:00.0,SAS,0,MIN,0,0,0,720.0,2880.0,31,19
2,1,11:44.0,SAS,0,MIN,2,1,0,704.0,2864.0,31,19
3,1,11:30.0,SAS,2,MIN,2,0,0,690.0,2850.0,31,19
4,1,11:11.0,SAS,2,MIN,2,0,0,671.0,2831.0,31,19


In [115]:
import pandas as pd
import joblib  # To load the saved model

# Define the time conversion function as provided
def convert_time_to_seconds(time_str):
    try:
        # The time format is like '12:00.0' (minute:second.millisecond)
        minutes, seconds = time_str.split(':')  # Split into minutes and seconds
        seconds, _ = seconds.split('.')  # Remove milliseconds (if any)
        return int(minutes) * 60 + int(seconds)
    except ValueError:
        # If there's an error in conversion, return NaN or handle it as needed
        return None

# Sample data (with 'Period' removed)
sample_data = pd.DataFrame({
    'Period': [1],
    'Time': ['12:00.0'],
    'AwayName': ['SAS'],
    'AwayScore': [0],
    'HomeName': ['MIN'],
    'HomeScore': [13]
})

# Convert 'Time' column to seconds and create 'TimeInSeconds' and 'TimeRemaining'
sample_data['TimeInSeconds'] = sample_data['Time'].apply(convert_time_to_seconds)
sample_data['TimeRemaining'] = (4 - sample_data['Period']) * 12 * 60 + sample_data['TimeInSeconds']

# Encoding AwayName and HomeName columns using the same LabelEncoder used during training
sample_data['AwayName_encoded'] = label_encoder.transform(sample_data['AwayName'])
sample_data['HomeName_encoded'] = label_encoder.transform(sample_data['HomeName'])

# Drop the original categorical columns and 'Period' (which is not used in the test)
sample_data_encoded = sample_data.drop(['AwayName', 'HomeName', 'Time', 'Period'], axis=1)

# Prepare the feature vector (X) for prediction
X_sample = sample_data_encoded



# Make predictions on the sample data
y_prob = model.predict_proba(X_sample)

# Extract the predicted probabilities for both HomeWin and AwayWin
predicted_home_win_prob = y_prob[0][:, 1]  # Probability for HomeWin class (1)
predicted_away_win_prob = y_prob[1][:, 1]  # Probability for AwayWin class (1)

# Output the predictions
print(f"Predicted HomeWin Probability: {predicted_home_win_prob[0]}")
print(f"Predicted AwayWin Probability: {predicted_away_win_prob[0]}")


Predicted HomeWin Probability: 0.8821006811904909
Predicted AwayWin Probability: 0.06637961682593735


In [106]:
joblib.dump(model, 'multi_output_model.pkl')  # Save the trained model
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [107]:
import os
os.listdir('/kaggle/working/')


['.virtual_documents', 'multi_output_model.pkl', 'label_encoder.pkl']

In [111]:
import shutil

# Move the file to a location in the working directory where it's easier to find
shutil.move('multi_output_model.pkl', '/kaggle/working/multi_output_model.pkl')


'/kaggle/working/multi_output_model.pkl'